In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.models as models

In [2]:
model = models.resnet18(pretrained=True)

/home/rohithk/miniconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rohithk/miniconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
train_transforms = transforms.Compose([
    transforms.ToTensor(),
])
test_transforms = transforms.Compose([
    transforms.ToTensor(),
])

In [4]:
train_dataset = datasets.CIFAR10(root='./data', train=True,transform=train_transforms, download=True)
test_dataset = datasets.CIFAR10(root='./data',train=False, transform=test_transforms, download=True)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=False)

In [6]:
epochs = 50
learning_rate = 0.0001

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [8]:
device

'cuda'

In [9]:
model = model.to(device)

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    model.train()
    r_loss = 0.0
    for images,labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs,labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        r_loss += loss
    print(f"Epoch: [{epoch + 1}/{epochs}], Loss: {r_loss / len(train_loader):.4f}")


Epoch: [1/50], Loss: 1.6223
Epoch: [2/50], Loss: 0.7003
Epoch: [3/50], Loss: 0.5203
Epoch: [4/50], Loss: 0.3901
Epoch: [5/50], Loss: 0.2898
Epoch: [6/50], Loss: 0.2260
Epoch: [7/50], Loss: 0.1784
Epoch: [8/50], Loss: 0.1360
Epoch: [9/50], Loss: 0.1112
Epoch: [10/50], Loss: 0.0990
Epoch: [11/50], Loss: 0.0884
Epoch: [12/50], Loss: 0.0827
Epoch: [13/50], Loss: 0.0705
Epoch: [14/50], Loss: 0.0675
Epoch: [15/50], Loss: 0.0610
Epoch: [16/50], Loss: 0.0591
Epoch: [17/50], Loss: 0.0534
Epoch: [18/50], Loss: 0.0488
Epoch: [19/50], Loss: 0.0481
Epoch: [20/50], Loss: 0.0464
Epoch: [21/50], Loss: 0.0441
Epoch: [22/50], Loss: 0.0418
Epoch: [23/50], Loss: 0.0393
Epoch: [24/50], Loss: 0.0363
Epoch: [25/50], Loss: 0.0359
Epoch: [26/50], Loss: 0.0398
Epoch: [27/50], Loss: 0.0313
Epoch: [28/50], Loss: 0.0313
Epoch: [29/50], Loss: 0.0306
Epoch: [30/50], Loss: 0.0314
Epoch: [31/50], Loss: 0.0292
Epoch: [32/50], Loss: 0.0283
Epoch: [33/50], Loss: 0.0268
Epoch: [34/50], Loss: 0.0278
Epoch: [35/50], Loss: 0

In [18]:
model.to("cpu")
model_path = "model.pth"
torch.save(model.state_dict(), model_path)
print(f"PyTorch model saved to {model_path}")

dummy_input = torch.randn(1, 3,32,32)  # Batch size of 1, input size of 10
model.eval()
# Convert to ONNX
onnx_path = "model.onnx"
torch.onnx.export(
    model,            # Model to be converted
    dummy_input,# Dummy input tensor
    onnx_path,               # Output ONNX file path
    export_params=True,      # Store the trained parameter weights inside the model
    opset_version=11,        # ONNX opset version to export to
    do_constant_folding=True, # Optimize constant folding for inference
    input_names=["input"],   # Input tensor name
    output_names=["output"], # Output tensor name
    
)

print(f"ONNX model saved to {onnx_path}")


PyTorch model saved to model.pth
ONNX model saved to model.onnx
